In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from typing import Callable
from celluloid import Camera

# 1D wave equation

$$
\frac{\partial ^{2} \phi }{\partial ^{2} t} -u^{2}\frac{\partial ^{2} \phi }{\partial ^{2} x} =0
$$

$$
\phi ( 0,t) =\phi ( L,t) =0,\quad \phi ( x,\ 0) =f( x) ,\quad \frac{\partial \phi ( x,\ 0)}{\partial t} =g( x)
$$

$$
\begin{pmatrix}
\phi _{1,j+1}\\
\phi _{2,j+1}\\
\vdots \\
\phi _{n-1,j+1}
\end{pmatrix} =\begin{pmatrix}
2\left( 1-\lambda ^{2}\right) & \lambda ^{2} & 0 & \cdots \\
\lambda ^{2} & 2\left( 1-\lambda ^{2}\right) & \lambda ^{2} & \cdots \\
\vdots  & \vdots  & \ddots  & \cdots \\
\cdots  & 0 & \lambda ^{2} & 2\left( 1-\lambda ^{2}\right)
\end{pmatrix}\begin{pmatrix}
\phi _{1,j}\\
\phi _{2,j}\\
\vdots \\
\phi _{n-1,j}
\end{pmatrix} -\begin{pmatrix}
\phi _{1,j-1}\\
\phi _{2,j-1}\\
\vdots \\
\phi _{n-1,j-1}
\end{pmatrix}
$$

$$
\phi _{i,1} =\left( 1-\lambda ^{2}\right) f( x_{i}) +\frac{\lambda ^{2}}{2}[ f( x_{i+1}) +f( x_{i-1})] +kg( x_{i})
$$

In [38]:
class Wave1d:
    def __init__(self, u: float, L: float, dx: float, dt: float, f, g):
        self.f = f
        self.g = g
        self.lamb = (u*dt/dx)**2
        N = int(L/dx)
        self.x = np.linspace(0, L, N)
        self.Y = [
            f(self.x[1:-1]),
            (1 - self.lamb)*f(self.x[1:-1]) + (self.lamb/2)*(f(self.x[2:]) + f(self.x[:-2])) + dt*g(self.x[1:-1])
        ]
        self.M = 2*(1 - self.lamb)*np.identity(N-2) + self.lamb*(np.diagflat(np.ones(N-3), -1) + np.diagflat(np.ones(N-3), 1))
    
    @property
    def y(self):
        return np.r_[0, self.Y[-1], 0]

    def step(self):
        y = self.M.dot(self.Y[-1]) - self.Y[-2]
        self.Y.append(y)
        return y

In [44]:
def show(animation):
    display(HTML(animation.to_html5_video()))
    plt.close()

In [45]:
wave = Wave1d(1, 1, 0.05, 0.01, lambda x: np.sin(np.pi*x), lambda x: 0)

camera = Camera(plt.gcf())
for _ in range(500):
    wave.step()
    plt.plot(wave.x, wave.y, 'C0-')
    camera.snap()
show(camera.animate(interval=5))

In [51]:
wave = Wave1d(1/(16*np.pi**2), 0.5, 0.005, 0.1, lambda x: 0, lambda x: np.sin(4*np.pi*x))

camera = Camera(plt.gcf())
for _ in range(1000):
    wave.step()
    plt.plot(wave.x, wave.y, 'C0-')
    camera.snap()
show(camera.animate(interval=5))

In [61]:
wave = Wave1d(1, 10, 0.05, 0.05, lambda x: np.where(x <= np.pi, np.sin(x), 0), lambda x: 0)

camera = Camera(plt.gcf())
for _ in range(1000):
    wave.step()
    plt.plot(wave.x, wave.y, 'C0-')
    camera.snap()
show(camera.animate(interval=10))